In [15]:
import pandas as pd
import numpy as np
import psycopg2
from sql import engine
import h3

pa_data = pd.read_csv('./data/Charging_PA.csv',sep=',')

/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (29,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [16]:
charging_df = pd.DataFrame(pa_data)

In [17]:
charging_df.drop(['Start Time Zone','End Time Zone', 'Transaction Date (Pacific Time)',
                'GHG Savings (kg)', 'Gasoline Savings (gallons)','Address 1','City', 
                'State/Province', 'Postal Code', 'Country', 'County', 'System S/N', 
                'Model Number','Org Name','EVSE ID','Energy (kWh)','Plug Type','Currency'], inplace= True, axis= 1)

In [18]:
charging_df.columns = charging_df.columns.str.replace(' ', '_')
charging_df.columns = charging_df.columns.str.lower()
charging_df.columns = charging_df.columns.str.rstrip()

In [19]:
charging_df = charging_df.rename(columns={'total_duration_(hh:mm:ss)': 'total_duration', "charging_time_(hh:mm:ss)": "charging_time"})

In [20]:
# Make datetime columns only for 2019
date_cols = ["start_date", "end_date"]
duration_cols = ["total_duration", "charging_time"]

for col in date_cols:
     charging_df[date_cols] = charging_df[date_cols].apply(pd.to_datetime, errors = "coerce")


for col in duration_cols:
     charging_df[duration_cols] = charging_df[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [21]:
charging_2020 = charging_df.query('start_date >= "2020-01-01" & start_date <= "2020-12-31"')
charging_2019 = charging_df.query('start_date >= "2019-01-01" & start_date <= "2019-12-31"')
charging_2018 = charging_df.query('start_date >= "2018-01-01" & start_date <= "2018-12-31"')
charging_2017 = charging_df.query('start_date >= "2017-01-01" & start_date <= "2017-12-31"')
charging_2016 = charging_df.query('start_date >= "2016-01-01" & start_date <= "2016-12-31"')
charging_2015 = charging_df.query('start_date >= "2015-01-01" & start_date <= "2015-12-31"')
charging_2014 = charging_df.query('start_date >= "2014-01-01" & start_date <= "2014-12-31"')
charging_2013 = charging_df.query('start_date >= "2013-01-01" & start_date <= "2013-12-31"')
charging_2012 = charging_df.query('start_date >= "2012-01-01" & start_date <= "2012-12-31"')
charging_2011 = charging_df.query('start_date >= "2011-01-01" & start_date <= "2011-12-31"')

In [22]:
charging_2019.head()

,station_name,mac_address,start_date,end_date,total_duration,charging_time,port_type,port_number,latitude,longitude,fee,ended_by,plug_in_event_id,driver_postal_code,user_id
191586,PALO ALTO CA / WEBSTER #3,0024:B100:0002:4277,2019-01-01 08:46:00,2019-01-01 10:58:00,0 days 02:11:14,0 days 02:06:39,Level 2,1,37.448273,-122.158234,2.86,Plug Out at Vehicle,1546361212,94601.0,492265.0
191587,PALO ALTO CA / BRYANT #6,0024:B100:0002:6B43,2019-01-01 09:18:00,2019-01-01 10:25:00,0 days 01:07:23,0 days 01:06:51,Level 2,1,37.446667,-122.162170,1.52,Plug Out at Vehicle,2951,94025.0,3174411.0
191588,PALO ALTO CA / BRYANT #1,000D:6F00:009D:9A1B,2019-01-01 09:19:00,2019-01-01 18:36:00,0 days 09:16:18,0 days 03:34:31,Level 2,2,37.446613,-122.162308,2.65,Plug Out at Vehicle,3524,94404.0,2358991.0
191589,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,2019-01-01 09:46:00,2019-01-01 11:35:00,0 days 01:49:47,0 days 01:49:26,Level 2,1,37.446690,-122.162231,1.40,Plug Out at Vehicle,2792,94025.0,555879.0
191590,PALO ALTO CA / HIGH #4,0024:B100:0002:1327,2019-01-01 09:54:00,2019-01-01 11:25:00,0 days 01:31:12,0 days 01:30:53,Level 2,2,37.443138,-122.163239,2.08,Plug Out at Vehicle,2416,94305.0,3171851.0


In [23]:
charging_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47655 entries, 191586 to 239240
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   station_name        47655 non-null  object         
 1   mac_address         47655 non-null  object         
 2   start_date          47655 non-null  datetime64[ns] 
 3   end_date            47655 non-null  datetime64[ns] 
 4   total_duration      47655 non-null  timedelta64[ns]
 5   charging_time       47655 non-null  timedelta64[ns]
 6   port_type           47655 non-null  object         
 7   port_number         47655 non-null  int64          
 8   latitude            47655 non-null  float64        
 9   longitude           47655 non-null  float64        
 10  fee                 47655 non-null  float64        
 11  ended_by            47638 non-null  object         
 12  plug_in_event_id    47655 non-null  int64          
 13  driver_postal_code  46251

In [24]:
# replace NaT in end_date with start_date + total_duration
charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration']))

<ipython-input-24-7ec4f59bfc9b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['end_date'] = charging_2019['end_date'].fillna((charging_2019['start_date'] + charging_2019['total_duration']))


In [25]:
# create column fee_bol (True if user paid, False if charging was free), drop original fee column
charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
charging_2019.drop(['fee'],inplace= True, axis= 1)

<ipython-input-25-99e212b80dbc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['fee_bol'] = np.where(charging_2019['fee']== 0.0, False, True)
/usr/local/Caskroom/miniconda/base/envs/sql-practice/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [26]:
# create colum parking_time to display parking time (plugged in but not charging)
charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']

<ipython-input-26-e1a37cd582e9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['parking_time'] = charging_2019['total_duration'] - charging_2019['charging_time']


Which year do we want to analyze?

unique identifier for each station plug = combination of plug ID and station name

In [27]:
# create column that displays weekday based on start_date
charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek

<ipython-input-27-202811ed20f3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['weekday'] = charging_2019['start_date'].dt.dayofweek


In [28]:
charging_2019['weekday'].unique()
mymapp = {'0': 'Monday', '1': 'Tuesday', '2': 'Wednesday', '3': 'Thursday', '4': 'Friday', '5': 'Saturday', '6': 'Sunday'}
charging_2019.applymap(lambda s: mymapp.get(s) if s in mymapp else s)

,station_name,mac_address,start_date,end_date,total_duration,charging_time,port_type,port_number,latitude,longitude,ended_by,plug_in_event_id,driver_postal_code,user_id,fee_bol,parking_time,weekday
191586,PALO ALTO CA / WEBSTER #3,0024:B100:0002:4277,2019-01-01 08:46:00,2019-01-01 10:58:00,0 days 02:11:14,0 days 02:06:39,Level 2,1,37.448273,-122.158234,Plug Out at Vehicle,1546361212,94601.0,492265.0,True,0 days 00:04:35,1
191587,PALO ALTO CA / BRYANT #6,0024:B100:0002:6B43,2019-01-01 09:18:00,2019-01-01 10:25:00,0 days 01:07:23,0 days 01:06:51,Level 2,1,37.446667,-122.162170,Plug Out at Vehicle,2951,94025.0,3174411.0,True,0 days 00:00:32,1
191588,PALO ALTO CA / BRYANT #1,000D:6F00:009D:9A1B,2019-01-01 09:19:00,2019-01-01 18:36:00,0 days 09:16:18,0 days 03:34:31,Level 2,2,37.446613,-122.162308,Plug Out at Vehicle,3524,94404.0,2358991.0,True,0 days 05:41:47,1
191589,PALO ALTO CA / BRYANT #5,0024:B100:0002:6D51,2019-01-01 09:46:00,2019-01-01 11:35:00,0 days 01:49:47,0 days 01:49:26,Level 2,1,37.446690,-122.162231,Plug Out at Vehicle,2792,94025.0,555879.0,True,0 days 00:00:21,1
191590,PALO ALTO CA / HIGH #4,0024:B100:0002:1327,2019-01-01 09:54:00,2019-01-01 11:25:00,0 days 01:31:12,0 days 01:30:53,Level 2,2,37.443138,-122.163239,Plug Out at Vehicle,2416,94305.0,3171851.0,True,0 days 00:00:19,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239236,PALO ALTO CA / WEBSTER #1,0024:B100:0002:4233,2019-12-30 18:57:00,2019-12-30 20:38:00,0 days 01:40:55,0 days 01:07:15,Level 2,1,37.448284,-122.158272,Plug Out at Vehicle,6046,95014.0,2248301,True,0 days 00:33:40,0
239237,PALO ALTO CA / HIGH #2,0024:B100:0002:5959,2019-12-30 18:58:00,2019-12-30 21:06:00,0 days 02:07:33,0 days 01:58:20,Level 2,2,37.443344,-122.163231,CPS Server,6126,94582.0,1691301,True,0 days 00:09:13,0
239238,PALO ALTO CA / BRYANT #3,000D:6F00:009E:D39E,2019-12-30 19:32:00,2019-12-31 07:37:00,0 days 12:05:36,0 days 09:40:19,Level 2,2,37.446583,-122.162109,Plug Out at Vehicle,13879,94301.0,713785,True,0 days 02:25:17,0
239239,PALO ALTO CA / CAMBRIDGE #1,0024:B100:0002:1329,2019-12-30 19:48:00,2019-12-31 08:00:00,0 days 12:11:45,0 days 11:05:21,Level 2,2,37.426155,-122.146065,Plug Out at Vehicle,10318,NaN,NaN,True,0 days 01:06:24,0


In [29]:
def make_weekday_names(row):
    if row['weekday'] == 0:
        return 'Monday'
    if row['weekday'] == 1:
        return 'Tuesday'
    if row['weekday'] == 2:
        return 'Wednesday'
    if row['weekday'] == 3:
        return 'Thursday'
    if row['weekday'] == 4:
        return 'Friday'
    if row['weekday'] == 5:
        return 'Saturday'
    if row['weekday'] == 6:
        return 'Sunday'

In [30]:
charging_2019['weekday'] = charging_2019.apply(lambda row : make_weekday_names(row), axis=1)

<ipython-input-30-2f77aeb8fa5c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['weekday'] = charging_2019.apply(lambda row : make_weekday_names(row), axis=1)


In [31]:
def make_mac_same(row):
    return row['mac_address'].replace(':','')

charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)

<ipython-input-31-ed39c61e56b2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['mac_address'] = charging_2019.apply(lambda row : make_mac_same(row), axis=1)


In [32]:
def make_unique_id(row):
    return str(row['mac_address']) + '_' +  str(row['port_number'])

charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)

<ipython-input-32-9e905a49289f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['unique_plug_id'] = charging_2019.apply(lambda row : make_unique_id(row), axis=1)


In [33]:
charging_2019['index1'] = charging_2019.index

<ipython-input-33-b41f60fb34ac>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['index1'] = charging_2019.index


In [34]:
# change order of columns, put unique_plug_id in front, drop MAC ID
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday']]

In [35]:
charging_2019 = charging_2019.sort_values(['unique_plug_id', 'start_date'])
charging_2019 = charging_2019.reset_index()
charging_2019.drop(['index'], inplace= True, axis= 1)
charging_2019['idle_time'] = None

In [36]:
max_index = int(len(charging_2019) -1)

for index, row in charging_2019.iterrows():
    if index >= max_index:
        charging_2019['idle_time'][index] = None
    elif index <= max_index:
        if charging_2019['unique_plug_id'][index+1] != charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = None
        elif charging_2019['unique_plug_id'][index+1] == charging_2019['unique_plug_id'][index]:
            charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]

<ipython-input-36-f94ba14cd3ef>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = charging_2019['start_date'][index+1] - charging_2019['end_date'][index]
<ipython-input-36-f94ba14cd3ef>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = None
<ipython-input-36-f94ba14cd3ef>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charging_2019['idle_time'][index] = None


In [37]:
duration_cols = ["idle_time"]

for col in duration_cols:
     charging_2019[duration_cols] = charging_2019[duration_cols].apply(pd.to_timedelta, errors="coerce")

In [38]:
# bring idle_time to front
charging_2019 = charging_2019[['station_name',
                            'unique_plug_id', 
                            'start_date',
                            'end_date',
                            'total_duration',
                            'charging_time',
                            'parking_time',
                            'idle_time',
                            'port_type',
                            'port_number',
                            'latitude',
                            'longitude',
                            'ended_by', 
                            'plug_in_event_id',
                            'driver_postal_code', 
                            'user_id',
                            'fee_bol', 
                            'weekday']]

### Potential Research Questions

Goal: Incentivize consumers in Germany (MUC and surroundings) to buy an EV by providing an optimal charging infrastructure (this means: there is (almost) always a charging station available at the right place (residential, leisure, work)

Identifying charging behavior patterns in relation to location specifics - Is Palo Alto a best practice example? If yes, what can other cities (e.g. MUC) learn from them in terms of charging infrastructure?

* Impact of COVID -- if time permits
* crisis 2013/2014 -- if time permits
* identify usage patterns according to location 
    * What are peak hours? Difference between weekdays / weekends?, per month? per year? 
    * peak times: How many PA users and how many from outside?
    * avg. charging time, avg. parking time, idle times
    * How many users are not from PA? What does MUC have to take into account in terms of "outside"/"shuttle" users?
    * Analyze recurring charging customers with zip codes not from PA? Which stations do they use? 
* What infrastructure strategy is PA using (e.g. how many new stations per year, where? (location characteristics); Is Palo Alto best practice? (define KPIs, what makes a best practice? Ideas: high idle times (even during peak hours or at least other free station nearby), high ratio of charging stations per EV, high coverage?)
* How does MUC compare?
* Which stations were added newly each year? compare with MUC

Potential categories: living, working, shopping, recreation

In [39]:
def make_h3_code(row):
    return h3.geo_to_h3(
            lat=row['latitude'],
            lng=row['longitude'],
            resolution=8    # Average Hexagon Area (km2): 0.7373276
                            # Average Hexagon Edge Length (km)
                            # https://h3geo.org/docs/core-library/restable/
)
charging_2019['h3_code'] = charging_2019.apply(lambda row : make_h3_code(row), axis=1)
charging_2019.h3_code.nunique()

5